# SSL/TLS handshake
An overview of SSL/TLS handshake performed with Python ssl library and Wireshark.

### Overview
SSL stands for **Secure Sockets Layer** and is designed to create secure connection between client and server. 
Secure means that connection is encrypted and therefore protected from eavesdropping. It also allows to validate server identity.

A secure connection can be assured through following means:
* Signed certificate, used to validate the server
* Private-public key encryption or key exchange performed during establishing the handshake
* Symmetric encryption of messages after handshake has been established

A protocol for establishing secure connection looks as follow:  
<img align="left" src="./media/handshake.png">

It consists of following steps:  
1. #### Client hello:
    - client version - versions of SSL/TLS protocols supported by the client
    - cipher suites - a list of cryptographic algorithms supported by the client
2. #### Server hello
    - server version - SSL/TLS version chosen by the server
    - cipher suites - cipher chosen by the server
    - certificate - signed certificate proving server's identity
    - server hello done - confirmation that server hello message is complete
3. #### Client key exchange:
    - key exchange
    - change cipher spec - information that any data sent afterwards will be encrypted
    - finished

### 1. Client hello
Examplary SSL/TLS connection can be performed using ssl module from Python's standard library:

In [1]:
import ssl
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ssl_sock = ssl.wrap_socket(sock, ciphers='ECDHE-RSA-AES128-SHA256')

ssl_sock.connect(('www.verisign.com', 443))

cipher, ssl_version, _ = ssl_sock.cipher()
print('Used cipher: {}, ssl version: {}'.format(cipher, ssl_version))

Used cipher: ECDHE-RSA-AES128-SHA256, ssl version: TLSv1/SSLv3


In this example, choice of **ECDHE-RSA-AES128-SHA256** cipher can be examined in a Client Hello message captured by Wireshark:

<img align="left" src="./media/client_hello.png">

### 2. Server hello
Server responds with a cipher version chosen from the list provided by the client. Usually it selects one of the strongest to ensure the best way to secure the connection, however since in this case client provided only one, it is the one being used.  
<img align="left" src="./media/server_hello.png">

In addition, server sends its certificate containing, among others, information about the issuer, certificate's expiry date and public key that can be used to encrypt pre-master secret key in RSA key-exchange method.  
<img align="left" src="./media/server_certificate.png">

Details regarding server's certificate can be extracted using **pyOpenSSL** library.

In [2]:
from datetime import datetime
from OpenSSL import crypto

pem_certificate = ssl.DER_cert_to_PEM_cert(ssl_sock.getpeercert(True))

certificate = crypto.load_certificate(crypto.FILETYPE_PEM, pem_certificate)
subject = certificate.get_subject()
issuer = certificate.get_issuer()

print('Subject:', subject.organizationName, subject.commonName)
print('Valid from:', datetime.strptime(certificate.get_notBefore().decode()[0:-1], '%Y%m%d%H%M%S'))
print('Valid to:', datetime.strptime(certificate.get_notAfter().decode()[0:-1], '%Y%m%d%H%M%S'))
print('Name of the issuer:', issuer.organizationName)

Subject: Verisign, Inc www.verisign.com
Valid from: 2017-08-02 00:00:00
Valid to: 2019-08-07 23:59:59
Name of the issuer: Symantec Corporation


After the client and server agree on a way of performing key exchange, another message is sent by the server (**Server Key Exchange**) containing information needed to calculate master key. Afterwards, the server ends its message with **Server Hello Done**.  
<img align="left" src="./media/server_key_exchange.png">

### 3. Client key exchange
After receiving all the information needed from the server, the client sends its details regarding key exchange. In this case its a public key that will be used by the server to compute master key allowing for symmetric encryption.  
<img align="left" src="./media/client_key_exchange.png">

**Change Cipher Spec** message informs that any data sent afterwards will be encrypted.

In [3]:
try:
    ssl_sock.sendall(b'GET / HTTP/1.1\r\nHost: www.verisign.com\r\nConnection: close\r\n\r\n')

    response = b''
    while True:
        data = ssl_sock.recv()
        if not data:
            break
        response += data

    response_lines = response.decode().splitlines()
    if not response_lines:
        print('No response received...')
    print('Protocol, response status: ', response_lines[0])
except (ConnectionError, ssl.SSLError) as e:
    print('Socket connection encountered an error: ', e)
ssl_sock.close()

Protocol, response status:  HTTP/1.1 200 OK


<img align="left" src="./media/application_data.png">